In [28]:
import wikipediaapi

secs = []

def get_sections(sections, level=0):
    for s in sections:
        if "references" not in s.title.lower() and "links" not in s.title.lower() and "gallery" not in s.title.lower():
        
            txt = s.text
            if txt != "": secs.append(txt)
        get_sections(s.sections, level + 1)

wiki_wiki = wikipediaapi.Wikipedia('en')


page = wiki_wiki.page("Artturi Ilmari Virtanen")

get_sections(page.sections)

wholePage = " ".join(s for s in secs)


In [29]:
import spacy
# You can use "en_core_web_md" or "en_core_web_sm", if you want
nlp = spacy.load("en_core_web_lg")

In [30]:
sents = []
for sent in nlp(wholePage).sents:
    sents.append(sent.text)
len(sents)

36

In [31]:
import numpy as np
from semantic_text_similarity.models import WebBertSimilarity

In [32]:
# You can use "cuda" instead of "cpu", if you have GPU
# I couldn't load the model in Windows, but in Linux the below line works just fine
web_model = WebBertSimilarity(device='cpu', batch_size=10)

In [33]:
# the following loops might take over an hour depending the computing power of your machine
simMatrix = []
for i, a in enumerate(sents):
    sentM = []
    for j, b in enumerate(sents):
        if i != j:
            sim = web_model.predict([(a,b)])[0]
            sentM.append(sim)
    simMatrix.append(sentM)
matrix = np.array(simMatrix)
len(matrix)

36

In [34]:
# save the numpy array to file..
# you might want to load the calculated array from the file later...
# https://numpy.org/doc/stable/reference/generated/numpy.save.html
with open("matrix.npy", "wb") as f:
    np.save(f, matrix)

In [35]:
matrix.shape

(36, 35)

In [36]:
from sklearn.cluster import KMeans

In [37]:
MAX_CLUSTERS = 4
# depending on your Python version, you might need to replace n_init=number, where number is 10 for example
kmeans = KMeans(n_clusters=MAX_CLUSTERS, random_state=42, init="random", n_init="auto").fit(matrix)

In [38]:
kmeans.labels_

array([3, 3, 3, 3, 3, 1, 1, 1, 3, 3, 2, 2, 2, 3, 2, 3, 3, 2, 1, 2, 1, 2,
       2, 1, 1, 1, 0, 1, 1, 1, 2, 0, 0, 0, 0, 0], dtype=int32)

In [39]:
len(kmeans.labels_)

36

In [40]:
index = [[k] for k in range(MAX_CLUSTERS)]

for k in range(MAX_CLUSTERS):
    for i, cluster in enumerate(kmeans.labels_):
        if k == cluster:
            index[k].append(i)
    index[k].pop(0)

In [41]:
index

[[26, 31, 32, 33, 34, 35],
 [5, 6, 7, 18, 20, 23, 24, 25, 27, 28, 29],
 [10, 11, 12, 14, 17, 19, 21, 22, 30],
 [0, 1, 2, 3, 4, 8, 9, 13, 15, 16]]

In [42]:
for k in range(MAX_CLUSTERS):
    print(f"\n== Cluster {k+1} ===\n")
    for i in index[k]:
        print(sents[i])


== Cluster 1 ===

In 1945, Virtanen received the Nobel Prize in chemistry "for his research and inventions in agricultural and nutrition chemistry".
The prestige conferred by the Nobel Prize brought Virtanen invitations, honorary doctorates and membership in foreign academies of science.
He was a member of the Finnish, Norwegian, Swedish, Flemish, Bavarian, and Pontifical Academies of Science, and of the Swedish and Danish Academies of Engineering Sciences.
He was an honorary member of learned societies in Finland, Sweden, Austria, Edinburgh, and the US, and received honorary degrees of the Universities of Lund, Paris, Giessen, and Helsinki, the Royal Technical College at Stockholm, and the Finland Institute of Technology.
The asteroid 1449 Virtanen, discovered by the renowned Finnish astronomer and physicist Yrjö Väisälä, was named after him.
The lunar crater Virtanen is also named after him.

== Cluster 2 ===

He saw in the overproduction of food only a temporary phenomenon.
He live